# AI-Enhanced Inventory Optimization
Forecast->EOQ/ROP/SS + Monte Carlo comparison.

In [ ]:
import pandas as pd, numpy as np
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

# Load or synthesize demand
try:
    df = pd.read_csv("../data/daily_demand.csv", parse_dates=["date"]).sort_values("date")
except FileNotFoundError:
    rng = pd.date_range("2023-01-01", periods=365, freq="D")
    demand = np.random.poisson(40, size=len(rng))
    df = pd.DataFrame({"date": rng, "demand": demand})

# Simple ML forecast (next-day demand)
df["lag1"] = df["demand"].shift(1).fillna(df["demand"].median())
X = df[["lag1"]].values
y = df["demand"].values
model = RandomForestRegressor(random_state=42).fit(X, y)
df["forecast"] = model.predict(X)

# Inventory parameters
lead_time_days = 5
service_factor = 1.65  # ~95%
demand_std = df["demand"].rolling(30).std().fillna(df["demand"].std())
daily_mu = df["forecast"]
# EOQ (placeholder constants)
K = 100.0   # order cost
h = 1.0     # holding cost per unit per day
annual_demand = df["demand"].sum()
EOQ = ((2 * K * annual_demand) / h) ** 0.5

# ROP & Safety Stock
mu_LT = daily_mu.rolling(lead_time_days).sum().fillna(daily_mu.mean()*lead_time_days)
sigma_LT = (demand_std * np.sqrt(lead_time_days)).fillna(demand_std.mean())
safety_stock = service_factor * sigma_LT
ROP = mu_LT + safety_stock

print("EOQ (units):", round(EOQ,2))
print(df[["date","forecast"]].tail())
